### OpenStreetMap Data Preprocessing - Extract Buildings Feature

In [2]:
#Mount google drive for content synchronization

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Install libraries for geospatial data wrangling and analysis

#Geopandas for wrangling
!pip install geopandas

#Rtree
!!apt install python3-rtree

#Contextily for basemap plotting 
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy
!pip3 install contextily==1.0rc2


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libproj-dev is already the newest version (4.9.3-2).
proj-bin is already the newest version (4.9.3-2).
proj-data is already the newest version (4.9.3-2).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgeos-dev is already the newest version (3.6.2-1build2).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


### Step 1: Load OSM data for Buildings

In [4]:
import geopandas
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame
import geopandas
from shapely.geometry import LineString, Point
import matplotlib.pyplot as plt
import contextily

building_df = geopandas.read_file('/content/drive/My Drive/Thesis/Bangladesh_OSM_Data/gis_osm_buildings_a_free_1.shp')
building_df.tail()

,osm_id,code,fclass,name,type,geometry
2825320,550319468,1500,building,None,None,"POLYGON ((88.57670 26.32947, 88.57671 26.32953..."
2825321,550319469,1500,building,None,None,"POLYGON ((88.57693 26.32935, 88.57693 26.32939..."
2825322,550319470,1500,building,None,None,"POLYGON ((88.57693 26.32940, 88.57694 26.32944..."
2825323,550319471,1500,building,None,None,"POLYGON ((88.57696 26.32938, 88.57696 26.32943..."
2825324,550319472,1500,building,None,None,"POLYGON ((88.57695 26.32932, 88.57696 26.32937..."


In [5]:
#Load cluster data
clusters_df = pd.read_csv('/content/drive/My Drive/Thesis/DHS_Data/bangladesh_cluster_avg_asset_2013_updated_new.csv')
clusters_df = geopandas.GeoDataFrame(clusters_df, geometry = geopandas.points_from_xy(clusters_df.longitude, clusters_df.latitude))
clusters_df.tail()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry
594,595,596,1.757480,U,24.900228,91.871489,POINT (91.87149 24.90023)
595,596,597,1.591590,U,24.904567,91.887165,POINT (91.88716 24.90457)
596,597,598,2.281590,U,24.886183,91.887103,POINT (91.88710 24.88618)
597,598,599,-0.042325,U,24.893073,91.906950,POINT (91.90695 24.89307)
598,599,600,1.242605,U,24.875934,91.895091,POINT (91.89509 24.87593)


OSM Building Data Feature Engineering 

In [6]:
#Build new dataframe the rows of which has info on types of building

building_df = building_df[building_df['type'].notna()]
building_df.tail()

,osm_id,code,fclass,name,type,geometry
2813598,550086967,1500,building,None,construction,"POLYGON ((88.73979 26.35772, 88.73979 26.35781..."
2816941,550099721,1500,building,None,construction,"POLYGON ((88.71573 26.34605, 88.71575 26.34617..."
2820578,550119938,1500,building,None,construction,"POLYGON ((88.73526 26.34509, 88.73526 26.34518..."
2821448,7858198,1500,building,None,house,"POLYGON ((91.90813 22.32515, 91.90821 22.32515..."
2824301,550240704,1500,building,None,hospital,"POLYGON ((90.44246 23.13809, 90.44282 23.13810..."


In [7]:
building_df.head()

,osm_id,code,fclass,name,type,geometry
0,23780888,1500,building,United Hospital,hospital,"POLYGON ((90.41523 23.80491, 90.41545 23.80516..."
13,24448713,1500,building,Bashundhara City Shopping Complex,commercial,"POLYGON ((90.38998 23.75087, 90.39018 23.75165..."
16,24463121,1500,building,East West Media Group Ltd.,commercial,"POLYGON ((90.43271 23.81336, 90.43458 23.81389..."
19,24820900,1500,building,Z.H. Sikder W.M.C. General and Cardiac Hospital,hospital,"POLYGON ((90.41998 23.79158, 90.42036 23.79162..."
22,28952569,1500,building,Concord IK Tower,commercial,"POLYGON ((90.41659 23.79545, 90.41696 23.79555..."


In [8]:
#Separate Urban and Rural area clusters

clusters_rural = clusters_df[clusters_df['URBAN_RURA'] == 'R']
clusters_urban = clusters_df[clusters_df['URBAN_RURA'] == 'U']
clusters_rural.head()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry
0,0,1,-0.831240,R,21.965697,90.126312,POINT (90.12631 21.96570)
1,1,2,-0.739180,R,22.221232,90.348908,POINT (90.34891 22.22123)
2,2,3,-0.786210,R,22.191484,90.212539,POINT (90.21254 22.19148)
3,3,4,-0.447090,R,22.185481,90.167023,POINT (90.16702 22.18548)
4,4,5,-0.687435,R,22.311173,90.149822,POINT (90.14982 22.31117)


In [9]:
clusters_urban.head()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry
50,50,51,0.145555,U,22.302611,90.091872,POINT (90.09187 22.30261)
51,51,52,0.913850,U,22.053795,89.968962,POINT (89.96896 22.05380)
52,52,53,-0.261730,U,22.813022,90.191408,POINT (90.19141 22.81302)
53,53,54,-0.231740,U,22.691392,90.389866,POINT (90.38987 22.69139)
54,54,55,1.585980,U,22.691265,90.640443,POINT (90.64044 22.69127)


In [10]:
type(clusters_rural.geometry[0])

shapely.geometry.point.Point

In [11]:
#convert Rural clusters to buffers with 5km radius
clusters_rural.crs = {'init' :'epsg:4326'}
clusters_rural= clusters_rural.to_crs(epsg=3174)  
buffer_length_in_meters = (5 * 1000)

rural_buffer = clusters_rural.geometry.buffer(buffer_length_in_meters)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [12]:
len(rural_buffer)

392

In [13]:
#convert Urban clusters to buffers with 2km radius
clusters_urban.crs = {'init' :'epsg:4326'}
clusters_urban= clusters_urban.to_crs(epsg=3174)  
buffer_length = (2 * 1000)

urban_buffer = clusters_urban.geometry.buffer(buffer_length)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [14]:
len(urban_buffer)

207

In [0]:
#Convert roads_df to same crs as Urban and Rural clusters df
building_df = building_df.to_crs(epsg=3174)

In [16]:
len(building_df)

46023

In [0]:
#Helper function to calculate the area of building type within a buffer zone of a cluster

def building_within(buffer,building_df): 
#  building_count_list = []
  building_area_total_list = []
  building_area_mean_list = []
  building_area_pct_list = []
  for point in buffer:
    building_area = building_df.intersection(point).area
    building_area_total = building_area.sum() / (1000**2)
    building_area_mean = building_area.mean() / (1000**2)
    building_area_pct = building_area_total / point.area 
    building_area_total_list.append(building_area_total)
    building_area_mean_list.append(building_area_mean)
    building_area_pct_list.append(building_area_pct)
  return building_area_total_list, building_area_mean_list, building_area_pct_list

In [0]:
#Helper function to calculate the number of building type within a buffer zone of a cluster

def building_count(buffer,building_df): 
  building_count_total = []
  for point in buffer:
    building_number = building_df.within(point).sum()
    building_count_total.append(building_number)
  return building_count_total

In [18]:
#Create list of different POIS types

building_type_name = building_df['type'].unique()
building_type_name

array(['hospital', 'commercial', 'train_station', 'industrial', 'office',
       'toll_booth', 'silo', 'gasometer', 'academic', 'university',
       'residential', 'other', 'Admin', 'BUET Admin', 'retail', 'school',
       'bank', 'dormatory', 'Canteen', 'auditorium', 'gallery',
       'me workshop', 'camp', 'vc', 'qk ext', 'canteen', 'post office',
       'hangar', 'mixed', 'apartments', 'WatchTower', 'house', 'mosque',
       'stadium', 'public', 'VC Residence', 'supermarket', 'garage',
       'dormitory', 'construction', 'garages', 'no',
       'English_Language_Cen', 'Chayanot,_cultural_c', 'hotel', 'church',
       'roof', 'college', 'College', 'temple', 'Union Council',
       'Orphanage', 'Gov._Office', 'Community_Center', 'Official', 'shed',
       'Goverment Office', 'cabin', 'detached', 'warehouse', 'hostel',
       'Tin shet', 'PAka', 'Katcha', 'apartment block', 'factory', 'new',
       'Tin-shade', 'health_clinic', 'kindergarten', 'under_construction',
       'residential,

In [19]:
#Create list of different POIS types

building_type_name = building_df['type'].unique()
building_type_name

array(['hospital', 'commercial', 'train_station', 'industrial', 'office',
       'toll_booth', 'silo', 'gasometer', 'academic', 'university',
       'residential', 'other', 'Admin', 'BUET Admin', 'retail', 'school',
       'bank', 'dormatory', 'Canteen', 'auditorium', 'gallery',
       'me workshop', 'camp', 'vc', 'qk ext', 'canteen', 'post office',
       'hangar', 'mixed', 'apartments', 'WatchTower', 'house', 'mosque',
       'stadium', 'public', 'VC Residence', 'supermarket', 'garage',
       'dormitory', 'construction', 'garages', 'no',
       'English_Language_Cen', 'Chayanot,_cultural_c', 'hotel', 'church',
       'roof', 'college', 'College', 'temple', 'Union Council',
       'Orphanage', 'Gov._Office', 'Community_Center', 'Official', 'shed',
       'Goverment Office', 'cabin', 'detached', 'warehouse', 'hostel',
       'Tin shet', 'PAka', 'Katcha', 'apartment block', 'factory', 'new',
       'Tin-shade', 'health_clinic', 'kindergarten', 'under_construction',
       'residential,

In [0]:
#Binning different values into one


building_df["type"].replace({"apartments": "residential", "dormatory": "residential", 'apartments': 'residential',\
                             'house': 'residential', 'dormitory': 'residential', 'apartment block': 'residential',\
                             'residential,shop,sch': 'residential', 'residential;commerci': 'residential','resedential': 'residential',
                             'hassan house': 'residential','VC Residence': 'residential', "commercial;residenti": 'residential', 'commercial;residenti': 'residential',
                             'apartment': 'residential','rrdidential':'residential', 'house 2': 'residential','three houses':'residential',
                             'Building_with_grocer':'residential', 'Building_with_laundr':'residential', 'House': 'residential',


                             'mosque': 'religious', 'cathedral': 'religious', 'Chayanot,_cultural_c': 'religious',\
                             'church':'religious', 'temple': 'religious', 'Mosque': 'religious',
                             

                             "Building_with_grocer": "commercial", 'cafe': 'commercial',\
                             'office': 'commercial', 'bank': 'commercial', 'hotel': 'commercial', 'retail': 'commercial',\
                             'warehouse': 'commercial', 'supermarket': 'commercial', 'store': 'commercial', 'Laboratory': 'commercial',
                             'Guesthouse_&_Confere': 'commercial', 'Hostel': 'commercial','11 Stored':'commercial', 'club':'commercial', 'shop':'commercial',
                             'market':'commercial', 'Shop': 'commercial', 'Sports_Building': 'commercial', 'CLC_Power_Company_Li': 'commercial',


                             'Stadium': 'sport', 'grandstand': 'sport','stadium': 'sport','arena': 'sport',


                             'silo': 'industrial', 'factory': 'industrial', 'storage_tank': 'industrial','manufacture': 'industrial',


                             'construction': 'construction', 'under_construction': 'construction', 'Temporary': 'construction',
                             'Semi permanent': 'construction', 'Under Construction':'construction', 'Underconstraction': 'construction',
                             'under construction': 'construction','non-permanent': 'construction', 'under_construction_1':'construction', 'semi-permanent':'construction',
                             'underconstruction': 'construction', 'Semi_permanent': 'construction', 'Semi_permanennt':'construction', 'Permanent':'construction', 'temporary':'construction', 'Semni_permanent': 'construction',

                             
                             "hospital": "health", 'medical': 'health', 'health_clinic': 'health', 'Clinic': 'health',


                             "train_station": "transport service", 'toll_booth': 'transport service', 'gasometer': 'transport service',\
                             'hangar': 'transport','WatchTower': 'transport', 'garage': 'transport', 'garages': 'transport',\
                             'transportation': 'transport', 


                             'university': 'education', 'BUET Admin':'education', 'Admin': 'education', 'school': 'education',\
                             'me workshop': 'education', 'canteen': 'education', 'English_Language_Cen': 'education',\
                             'College': 'education', 'college': 'education','kindergarten': 'education', 'Madrasha': 'education',
                             'madrasa': 'education', 'educational_institut': 'education', 'Recreational': 'education', 'Cantin': 'education',
                             'Nawab_Faizunnesa_Hal': 'education','Madrasa': 'education', 'School': 'education', 'Nawab_faizunnesa_Hal': 'education', 'Gopalpur High School': 'education', 'Institution': 'education',
                             'academic': 'education',

                             'public':'government', 'service': 'government', 'Union Council': 'government', 'Community_Center': 'government',\
                             'Official':'government', 'Goverment Office': 'government', 'Police_Station': 'government', 'goverment': 'government',
                             'infrastructure': 'government','civil_service': 'government','Police_station': 'government', 'civic': 'government',


                             "Hut": 'low-cost', 'Orphanage': 'low-cost', 'shed': 'low-cost', 'cabin': 'low-cost',\
                              'Tin shet': 'low-cost', 'Tin-shade': 'low-cost', 'tin shade house':'low-cost', 'Tin Shed House':'low-cost', 'Tin shaded':'low-cost', 'tin shaded':'low-cost',
                              'hut': 'low-cost', 'Brick_Kiln': 'low-cost', 'Low Income Settlemen': 'low-cost', 'Slum': 'low-cost',
                             'kiosk': 'low-cost', 'Tin Shed':"low-cost",'slum': 'low-cost','Garbage Area':'low-cost', 'Tinshed':'low-cost','stable': 'low-cost',
                             'Tin_shed':'low-cost', 'tin_shed':'low-cost','Tinshed building':'low-cost', 'tinshed': 'low-cost', 'hamlet': 'low-cost','ruins': 'low-cost', 'bungalow': 'low-cost',


                             'Agriculture': 'agriculture', 'barn': 'agriculture', 'farm': 'agriculture', 'farm_auxiliary': 'agriculture', 'greenhouse?':'agriculture',
                             'greenhouse': 'agriculture', 'cowshed': 'agriculture',


                             'mixed': 'other', 'no': 'other', 'roof': 'other', 'PAka': 'other', 'Katcha': 'other', 'new': 'other', 'detached': 'other', 'yes4': 'other', 'unclassified': 'other', 'Cart': 'other',
                             'mix': 'other', 'mixed use': 'other', 'godown': 'other', 'car': 'other','Multipurpose': 'other', 'No':'other', 'no house':'other',
                             'unused':'other', 'sculpture':'other', 'pond':'other', 'risesn':'other', 'wood':'other', 'terrace':'other', 'place':'other', 'uy':'other', 'Marker':'other', 'well':'other', 
                             '3': 'other', 'q': 'other','\\': 'other', 'ola_del_mar':'other','yesS':'other','toilet':'other', 'CO': 'other', 'C': 'other', 'yesqq': 'other', 'yeb': 'other', 'B-49': 'other','333': 'other','yesq': 'other', 'Need a plan': 'other'
                        
                             }, inplace=True)


In [21]:
building_type_name = building_df['type'].unique()
building_type_name

array(['health', 'commercial', 'transport service', 'industrial',
       'education', 'residential', 'other', 'Canteen', 'auditorium',
       'gallery', 'camp', 'vc', 'qk ext', 'post office', 'transport',
       'religious', 'sport', 'government', 'construction', 'low-cost',
       'Gov._Office', 'hostel', 'residential,commerci', 'agriculture',
       'water', 'residential_cum_comm', 'Semi-Pucca', 'shed;residential',
       'Residential_cum_Comm', 'land', 'residential, Commerc', 'building',
       'residential_&_commer', 'rrsidential', 'residean',
       'Medical center', 'residence', 'Yes', 'Kashipur, Barisal'],
      dtype=object)

In [0]:
building_df["type"].replace({'Canteen': 'education', 'auditorium': 'sport', 'gallery': 'sport', 'camp': 'sport', 'vc': 'other',
                             'qk ext': 'other', 'post office': 'government', 'Gov._Office': 'government', 'hostel': 'commercial', 
                             'residential,commerci': 'residential', 'water': 'other', 'residential_cum_comm': 'residential', 
                             'Semi-Pucca': 'low-cost', 'shed;residential': 'low-cost', 'Residential_cum_Comm': 'residential',
                             'land': 'other', 'residential, Commerc': 'residential', 'building': 'other', 'residential_&_commer': 'residential', 
                             'rrsidential': 'residential', 'residean': 'residential', 'Medical center': 'health', 'residence': 'residential', 
                             'Yes': 'other', 'Kashipur, Barisal': 'other'}, inplace=True)

In [23]:
building_type_name = building_df['type'].unique()
building_type_name

array(['health', 'commercial', 'transport service', 'industrial',
       'education', 'residential', 'other', 'sport', 'government',
       'transport', 'religious', 'construction', 'low-cost',
       'agriculture'], dtype=object)

In [0]:
#Create list of dataframes of different building types
building_type = []
for i in range(len(building_type_name)): 
  building_type_list = building_df[building_df['type']== building_type_name[i]]
  building_type.append(building_type_list) 

In [0]:
##Engineer the total area of building feature for each types and append to rural dataframe

for i in range(len(building_type)):
  building_area_total_list, building_area_mean_list, building_area_pct_list = building_within(rural_buffer, building_df)
  building_area_total_list = np.asarray(building_area_total_list)
  building_area_mean_list = np.asarray(building_area_mean_list)
  building_area_pct_list = np.asarray(building_area_pct_list)
  clusters_rural[f"{building_type_name[i]} building area total"] = building_area_total_list
  clusters_rural[f"{building_type_name[i]} building area mean"] = building_area_mean_list
  clusters_rural[f"{building_type_name[i]} building area proportion"] = building_area_pct_list  

In [0]:
##Engineer the total area of building feature for each types and append to urban dataframe

for i in range(len(building_type)):
  building_area_total_list, building_area_mean_list, building_area_pct_list = building_within(urban_buffer, building_df)
  building_area_total_list = np.asarray(building_area_total_list)
  building_area_mean_list = np.asarray(building_area_mean_list)
  building_area_pct_list = np.asarray(building_area_pct_list)
  clusters_urban[f"{building_type_name[i]} building area total"] = building_area_total_list
  clusters_urban[f"{building_type_name[i]} building area mean"] = building_area_mean_list
  clusters_urban[f"{building_type_name[i]} building area proportion"] = building_area_pct_list  

In [0]:
##Engineer the total area of building feature for each types and append to rural dataframe

#get counts: https://stackoverflow.com/questions/54127731/how-can-i-count-the-number-of-polygons-a-shape-intersects

import pandas as pd
import geopandas as gp
from shapely.geometry import Polygon
from shapely.geometry import Point
import matplotlib.pyplot as plt

rural_buffer_copy = rural_buffer.copy()

# generate spatial index
sindex = building_df.sindex
# define empty list for results
results_list = []
# iterate over the points
for buffer in rural_buffer_copy: 
    # find approximate matches with r-tree, then precise matches from those approximate ones
    possible_matches_index = list(sindex.intersection(buffer.bounds))
    possible_matches = building_df.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(buffer)]
    results_list.append(len(precise_matches))

# add list of results as a new column
rural_buffer_copy['polygons'] = pd.Series(results_list)

# for i in range(len(building_type)):
#   building_count_total = building_count(rural_buffer, building_df)
#   building_count_total = np.asarray(building_count_total)
#   clusters_rural[f"Count of {building_type_name[i]} building"] = building_count_total


/usr/local/lib/python3.6/dist-packages/geopandas/base.py:104: UserWarning: Cannot generate spatial index: Missing package `rtree`.
  warn("Cannot generate spatial index: Missing package `rtree`.")


AttributeError: ignored

In [0]:
building_df_copy = building_df.copy()
#building_df_copy = gp.GeoDataFrame(building_df_copy, geometry = 'geometry')

sindex = building_df_copy.sindex
print(sindex)

None


/usr/local/lib/python3.6/dist-packages/geopandas/base.py:104: UserWarning: Cannot generate spatial index: Missing package `rtree`.
  warn("Cannot generate spatial index: Missing package `rtree`.")


In [0]:
##Engineer the total area of building feature for each types and append to rural dataframe



for i in range(len(building_type)):
  building_count_total = building_count(urban_buffer, building_df)
  building_count_total = np.asarray(building_count_total)
  clusters_urban[f"Count of {building_type_name[i]} building"] = building_count_total


# New Section

In [0]:
import os
import pickle
os.chdir('/content/drive/My Drive/Thesis/OSM_data')
with open('clusters_rural_building', 'wb') as fp:
    pickle.dump(clusters_rural, fp)

In [0]:
import os
import pickle
os.chdir('/content/drive/My Drive/Thesis/OSM_data')
with open('clusters_urban_building', 'wb') as fp:
    pickle.dump(clusters_urban, fp)

In [0]:
with open('clusters_rural_building', 'rb') as fp:
  clusters_rural = pickle.load(fp)

with open('clusters_urban_building', 'rb') as fp:
  clusters_urban = pickle.load(fp)

# New Section

### Merge engineered features with the original data


In [0]:
merged = pd.concat([clusters_rural, clusters_urban])
merged = merged.sort_values(by = 'cluster')

In [0]:
with open('merged_building', 'wb') as fp:
    pickle.dump(merged, fp)

In [0]:
merged.to_csv('clusters_building.csv')